***
This notebook is inteded to integrate and clean the project data
***

In [1]:
import os 
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=3g  pyspark-shell"
from pyspark.sql import SparkSession
try: 
    spark
    print("Spark application already started. Terminating existing application and starting new one")
    spark.stop()
except: 
    pass
# Create a new spark session (note, the * indicates to use all available CPU cores)
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("H600 L-Group") \
    .getOrCreate()
#When dealing with RDDs, we work the sparkContext object. See https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext
sc=spark.sparkContext
#in local mode, you will be able to access the Spark GUI at http://localhost:4040

## 1: Integration
-If a column exists in the old schema version, but not in the new one, drop its contents.

-If a column exists both in the old schema version and the new (but possibly under a
different name, or the order of the column in the new schema version is different), copy
the value.

-If a column does not exist in the old, but does exist in the new, give it a blank value
(indicating that the record does not have such a value).

-If a column does not exists in the old schema, but does exist in the new, and it is possible
to compute the value of the new column given information in the old schema, then all in
this computed value.


In [8]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
sql_c = SQLContext(sc)

In [14]:
#visualize a specific file
df_green = sql_c.read.csv('testspark/fhvhv_tripdata_2019-02.csv', header=True)
df_green.show(3)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0002|              B02914|2019-02-25 15:29:31|2019-02-25 15:42:55|         161|         164|   null|
|           HV0004|              B02800|2019-02-12 07:21:27|2019-02-12 07:59:23|         255|         170|      1|
|           HV0003|              B02878|2019-02-23 09:16:03|2019-02-23 09:32:50|         168|         243|   null|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
only showing top 3 rows



In [28]:
#visualize header
df_green.columns[:7]

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [26]:
#visualize only specific columns in a specific order
df_test3column=df_green.select('pickup_datetime','PULocationID','SR_Flag').show(3)

#try to export that visu into a new csv file
#df_test3column.write.format('com.databricks.spark.csv').save('testspark/modified/3columns.csv')
#df_test3column.save('testspark/modified/3columns.csv', 'com.databricks.spark.csv')
#df_test3column.write.csv('testspark/modified/3columns.csv')

+-------------------+------------+-------+
|    pickup_datetime|PULocationID|SR_Flag|
+-------------------+------------+-------+
|2019-02-25 15:29:31|         161|   null|
|2019-02-12 07:21:27|         255|      1|
|2019-02-23 09:16:03|         168|   null|
+-------------------+------------+-------+
only showing top 3 rows



In [29]:
#drop a column
df_green.select('pickup_datetime','PULocationID').drop('SR_Flag').show(3)
#df_test3column.drop('pickup_datetime').collect(3)
df_green.columns[:7]
#SR8Flag is still there

+-------------------+------------+
|    pickup_datetime|PULocationID|
+-------------------+------------+
|2019-02-25 15:29:31|         161|
|2019-02-12 07:21:27|         255|
|2019-02-23 09:16:03|         168|
+-------------------+------------+
only showing top 3 rows



['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [ ]:


fileName = 'Data/green_tripdata_2020-01.csv'
GreenRDD = sc.textFile(fileName)
GreenRDD.take(5)
df = GreenRDD
df.drop('congestion_surcharge')
#drop column:
#    df = dataframe
#    df.drop('age').collect()
#    
#change order:
#    https://www.datasciencemadesimple.com/re-arrange-or-re-order-column-in-pyspark/#:~:text=In%20order%20to%20Rearrange%20or,rearrange%20the%20column%20by%20position.
#    
#add column:
#    df.withColumn("location", lit())